# Turn deck lists into useful format

In [76]:
import requests
import json

#### Scrape some deck lists to work with

In [3]:
deck_lists = scrape_deck_lists()

Requesting the front pages
	Requesting front page number 0
Getting all event ids
	Getting event ids from a front page
Getting deck lists
		Event request for event id 18141
		Event request for event id 18048
		Event request for event id 18142
		Event request for event id 18129
		Event request for event id 18128
		Event request for event id 18060
		Event request for event id 18059
		Event request for event id 18072
		Event request for event id 18074
		Event request for event id 18143
		Deck request for deck id 312465
		Deck request for deck id 312461
		Deck request for deck id 312454
		Deck request for deck id 312389
		Deck request for deck id 312031
		Deck request for deck id 312141
		Deck request for deck id 312395
		Deck request for deck id 312394
		Deck request for deck id 312482
		Deck request for deck id 312144
		Deck request for deck id 312391
		Deck request for deck id 312021
		Deck request for deck id 312393
		Deck request for deck id 312035
		Deck request for deck id 312034
		D

In [18]:
deck_lists.keys()

TypeError: 'dict_keys' object does not support indexing

In [19]:
sample_deck_list = deck_lists[list(deck_lists.keys())[0]]

#### Get all modern legal cards (i think all)

In [7]:
total_pages = 0
url = 'https://api.scryfall.com/cards/search?q=format:modern'
modern_legal_cards_scryfall = []
while True:
    response = requests.get(url, headers={'User-Agent': 'Getting modern legal cards'})
    
    if response.status_code != 200:
        print('Bad Status code on page #{}: {}'.format(response.status_code))
        break
        
    cards = json.loads(response.text)
    
    if not cards['has_more']:
        print('All done! {} pages recieved.'.format(total_pages))
        break
    
    modern_legal_cards_scryfall.append(cards['data'])
    total_pages += 1
    url = cards['next_page']
    
    time.sleep(0.05 + random.random())
    
    print('Page request #{} successful! Onto the next.'.format(total_pages))

Page request #1 successful! Onto the next.
Page request #2 successful! Onto the next.
Page request #3 successful! Onto the next.
Page request #4 successful! Onto the next.
Page request #5 successful! Onto the next.
Page request #6 successful! Onto the next.
Page request #7 successful! Onto the next.
Page request #8 successful! Onto the next.
Page request #9 successful! Onto the next.
Page request #10 successful! Onto the next.
Page request #11 successful! Onto the next.
Page request #12 successful! Onto the next.
Page request #13 successful! Onto the next.
Page request #14 successful! Onto the next.
Page request #15 successful! Onto the next.
Page request #16 successful! Onto the next.
Page request #17 successful! Onto the next.
Page request #18 successful! Onto the next.
Page request #19 successful! Onto the next.
Page request #20 successful! Onto the next.
Page request #21 successful! Onto the next.
Page request #22 successful! Onto the next.
Page request #23 successful! Onto the nex

In [9]:
card_dict = {}
for page in modern_legal_cards_scryfall:
    for card in page:
        # 'id' is scryfalls unique ID
        card_dict[card['name']] = card['id']

There should be 11,348 cards in card_dict (as of 1/15/18) according to scryfall.

    Although, I only get 11,200 as of 1/15/18

In [12]:
len(card_dict)

11200

In [11]:
card_dict['Kumena, Tyrant of Orazca']

'a3aef818-c896-46e6-aaff-56aee52a066c'

In [20]:
sample_deck_list

['4 Tectonic Edge',
 '2 Shefet Dunes',
 '8 Plains',
 '1 Mutavault',
 '4 Horizon Canopy',
 '4 Ghost Quarter',
 '1 Flagstones of Trokair',
 '4 Aether Vial',
 '4 Path to Exile',
 '4 Thraben Inspector',
 '4 Thalia, Guardian of Thraben',
 '2 Serra Avenger',
 '2 Selfless Spirit',
 '3 Restoration Angel',
 '4 Leonin Arbiter',
 '4 Flickerwisp',
 '4 Blade Splicer',
 '1 Aven Mindcensor',
 'Sideboard',
 '1 Sunlance',
 '2 Stony Silence',
 '1 Settle the Wreckage',
 '1 Rest in Peace',
 '1 Relic of Progenitus',
 '2 Oblivion Ring',
 '2 Mirran Crusader',
 "1 Grafdigger's Cage",
 '1 Eidolon of Rhetoric',
 '1 Burrenton Forge-Tender',
 '1 Selfless Spirit',
 '1 Aven Mindcensor',
 '']

In [29]:
card_nums = []
card_names = []
for row in sample_deck_list:
    if row[0] == 'S':
        break
    card_nums.append(int(row[0]))
    card_names.append(row[1:])

In [30]:
card_nums

[4, 2, 8, 1, 4, 4, 1, 4, 4, 4, 4, 2, 2, 3, 4, 4, 4, 1]

In [31]:
card_names

[' Tectonic Edge',
 ' Shefet Dunes',
 ' Plains',
 ' Mutavault',
 ' Horizon Canopy',
 ' Ghost Quarter',
 ' Flagstones of Trokair',
 ' Aether Vial',
 ' Path to Exile',
 ' Thraben Inspector',
 ' Thalia, Guardian of Thraben',
 ' Serra Avenger',
 ' Selfless Spirit',
 ' Restoration Angel',
 ' Leonin Arbiter',
 ' Flickerwisp',
 ' Blade Splicer',
 ' Aven Mindcensor']

In [48]:
temp = {}
user_card_counts = []
for deck_id, deck_list in deck_lists.items():
    card_counts = []
    card_names = []
    for row in sample_deck_list:
        # not worrying about sideboards
        if row[0] == 'S':
            break
        card_count = int(row[0])
        card_name = row[2:]
        user_card_counts.append((int(deck_id), card_name, card_count))
        
        
    temp[int(deck_id)] = {'card_counts': card_counts, 'card_names': card_names}

In [51]:
len(user_card_counts)

1224

In [58]:
with open('user_card_counts.json', 'w') as f:
    json.dump(user_card_counts, f)

In [59]:
import boto3

In [72]:
s3 = boto3.resource('s3')
s3_object = s3.Object('mtg-capstone', 'user_card_count.json')

In [73]:
s3_object.upload_file('user_card_counts.json')

In [77]:
import deck_scraping
import card_scraping
import make_user_card_counts

In [114]:
deck_lists = deck_scraping.scrape_deck_lists(verbose=False)

In [103]:
user_card_counts = make_user_card_counts.make_user_card_counts(deck_lists)

user_card_counts

[(312465, 'Tectonic Edge', 4),
 (312465, 'Shefet Dunes', 2),
 (312465, 'Plains', 8),
 (312465, 'Mutavault', 1),
 (312465, 'Horizon Canopy', 4),
 (312465, 'Ghost Quarter', 4),
 (312465, 'Flagstones of Trokair', 1),
 (312465, 'Aether Vial', 4),
 (312465, 'Path to Exile', 4),
 (312465, 'Thraben Inspector', 4),
 (312465, 'Thalia, Guardian of Thraben', 4),
 (312465, 'Serra Avenger', 2),
 (312465, 'Selfless Spirit', 2),
 (312465, 'Restoration Angel', 3),
 (312465, 'Leonin Arbiter', 4),
 (312465, 'Flickerwisp', 4),
 (312465, 'Blade Splicer', 4),
 (312465, 'Aven Mindcensor', 1),
 (312461, 'Thalia, Guardian of Thraben', 3),
 (312461, 'Unclaimed Territory', 4),
 (312461, 'Godless Shrine', 1),
 (312461, "Avacyn's Pilgrim", 3),
 (312461, 'Plains', 1),
 (312461, 'Noble Hierarch', 4),
 (312461, 'Reflector Mage', 4),
 (312461, "Thalia's Lieutenant", 4),
 (312461, 'Windswept Heath', 4),
 (312461, 'Temple Garden', 1),
 (312461, 'Meddling Mage', 3),
 (312461, 'Forest', 1),
 (312461, 'Kitesail Freebooter

In [109]:
with open('data/user_card_counts2.json', 'w') as f:
    json.dump(user_card_counts, f)

In [111]:
s3 = boto3.client('s3')

In [112]:
filename = 'data/user_card_counts2.json'
bucketname = 'mtg-capstone'


In [113]:
s3.upload_file(filename, bucketname, filename)